In [1]:
import numpy as np

# Decision Tree Node

In [2]:
class DecisionTreeNode:
    # chose-able feature in this node, 1 is chose-able
    features = []
    # chosen feature id
    feature_id = -1
    # chosen feature name
    feature_names = []
    feature_name = ''
    # the data needs to make decision in this node
    data = None
    data_left = None
    data_right = None
    # the entrophy of the self.data
    entrophy = 1
    # the entrophy of split data
    entrophy_sub = 1
    # node depth
    depth = 1
    # node type 0:mid\ 1:leaf
    node_type = 0
    # threshold
    threshold = 0.05
    max_depth = 5
    # sub nodes
    node_left = None
    node_right = None
    # for leaf node
    result = -1

    def H(self, x: float):
        """
        calculate the entropy of the number
        :param x: the positive ratio
        :return: the H(x), entropy of x
        """
        if x == 0.0 or x == 1.0:
            return 0
        h = - x * np.log2(x) - (1 - x) * np.log2(1 - x)
        return h

    def calculate_sub_entrophy(self, data_pos, data_neg) -> float:
        """
        calculate the entrophy of spliting by a feature
        :param data_pos: pos data of the feature
        :param data_neg: neg data of the feature
        :return: entrophy
        """
        pos_pos = 0
        neg_pos = 0
        pos_amount = len(data_pos)
        neg_amount = len(data_neg)
        total_amount = pos_amount + neg_amount
        wp = pos_amount / total_amount
        wn = neg_amount / total_amount
        for data_id in range(pos_amount):
            if data_pos[data_id][-1] == 1:
                pos_pos += 1
        for data_id in range(neg_amount):
            if data_neg[data_id][-1] == 1:
                neg_pos += 1
        if pos_amount == 0:
            p1 = 0
        else:
            p1 = pos_pos / pos_amount
        if neg_amount == 0:
            p2 = 0
        else:
            p2 = neg_pos / neg_amount
        entrophy = wp * self.H(p1) + wn * self.H(p2)
        return entrophy

    def choose_feature(self):
        """
        Choose the best feature to split the data in this node.
        :update:
                - self.features
                - self.feature_id
                - self.feature_name
                - self.entrophy_sub
        :return: None
        """
        if self.entrophy == 0:
            self.result = self.data[0][-1]
            self.node_type = 1
            return
        feature_len = len(self.features)
        data_len = len(self.data)
        entrophy_min = 1
        chosen_feature_id = -1
        data_left = None
        data_right = None
        for feature_id in range(feature_len):
            # feature haven't been used yet
            if self.features[feature_id] == 1:
                data_neg = []
                data_pos = []
                for data_id in range(data_len):
                    if self.data[data_id][feature_id] == 0:
                        data_neg.append(self.data[data_id])
                    else:
                        data_pos.append(self.data[data_id])
                entrophy = self.calculate_sub_entrophy(data_pos, data_neg)
                # update the chosen feature
                if entrophy < entrophy_min:
                    chosen_feature_id = feature_id
                    entrophy_min = entrophy
                    data_left = data_pos
                    data_right = data_neg
        self.data_left = data_left
        self.data_right = data_right
        self.feature_id = chosen_feature_id
        if self.feature_id >= 0:
            self.feature_name = self.feature_names[chosen_feature_id]
        self.features[chosen_feature_id] = 0
        self.entrophy_sub = entrophy_min
        information_gain = self.entrophy - entrophy_min
        if information_gain <= self.threshold:
            # print('information gain is lower than threshold!\n'
            #       f'information gain = {information_gain}\n'
            #       f'threshold = {self.threshold}\n'
            #       f'setting the node to leaf')
            self.node_type = 1
            # here may exist bug
            self.result = self.data[0][-1]

    def calculate_self_entrophy(self) -> float:
        """
        calculate the entrophy, base on self data
        :return: entrophy
        """
        data_amount = len(self.data)
        pos_count = 0
        for data_id in range(data_amount):
            if self.data[data_id][-1] == 1:
                pos_count += 1
        p = pos_count / data_amount
        entrophy = self.H(p)
        self.entrophy = entrophy
        return entrophy

    def split(self):
        self.node_left = None
        self.node_right = None
        if self.depth == self.max_depth \
                or \
                self.node_type == 1:
            if self.depth == self.max_depth:
                pass
                # print(
                #     f"depth reach max_depth!\n"
                #     f"depth = {self.depth}\n"
                #     f"setting the type to leaf"
                # )
            del self.data
            return self.node_left, self.node_right
        self.node_left = DecisionTreeNode(
            data=self.data_left,
            features=self.features,
            feature_names=self.feature_names,
            threshold=self.threshold,
            max_depth=self.max_depth,
            depth=self.depth + 1
        )
        self.node_right = DecisionTreeNode(
            data=self.data_right,
            features=self.features,
            feature_names=self.feature_names,
            threshold=self.threshold,
            max_depth=self.max_depth,
            depth=self.depth + 1
        )
        del self.data
        del self.data_left
        del self.data_right
        return self.node_left, self.node_right

    def info(self):
        print(
            f"data:\n{self.data}\n"
        )

    def __init__(self, data, features, feature_names, threshold=0.05, max_depth=10, depth=1):
        self.features = np.copy(features)
        self.feature_names = feature_names
        self.depth = depth
        self.threshold = threshold
        self.max_depth = max_depth
        self.data = data
        self.calculate_self_entrophy()
        self.choose_feature()

    def __str__(self):
        if self.node_type == 0:
            return f"feature: {self.feature_name}"
        else:
            return f"result: {self.result}"

# DFS & BFS function
1. DFS
    to regression generate the whole tree
2. BFS
    to print the result

In [3]:
def dfs(node: DecisionTreeNode):
    if node == None:
        return
    node_left, node_right = node.split()
    # print(node, node_left, node_right)
    dfs(node_left)
    dfs(node_right)

In [4]:
def bfs(root: DecisionTreeNode):
    if root == None:
        return
    queue = []
    queue.append(root)
    while len(queue) != 0:
        size = len(queue)
        while size > 0:
            size -= 1
            node = queue.pop(0)
            print(node)
            if node.node_left != None:
                queue.append(node.node_left)
            if node.node_right != None:
                queue.append(node.node_right)
        print()

# Cat classification example

| Ear shape(Pointy) | Face shape(Round) | Whickers(Present) | Cat  |
| ----------------- | ----------------- | ----------------- | ---- |
| 1                 | 1                 | 1                 | 1    |
| 0                 | 0                 | 1                 | 1    |
| 0                 | 1                 | 0                 | 0    |
| 1                 | 0                 | 1                 | 0    |
| 1                 | 1                 | 1                 | 1    |
| 1                 | 1                 | 0                 | 1    |
| 0                 | 0                 | 0                 | 0    |
| 1                 | 1                 | 0                 | 1    |
| 0                 | 1                 | 0                 | 0    |
| 0                 | 1                 | 0                 | 0    |



In [5]:
data = np.array([
    [1, 1, 1, 1],
    [0, 0, 1, 1],
    [0, 1, 0, 0],
    [1, 0, 1, 0],
    [1, 1, 1, 1],
    [1, 1, 0, 1],
    [0, 0, 0, 0],
    [1, 1, 0, 1],
    [0, 1, 0, 0],
    [0, 1, 0, 0],
], dtype=int)
feature_name = [
    'Ear shape',
    'Face shape',
    'Whiskers'
]
root = DecisionTreeNode(
    data=data,
    features=[1, 1, 1],
    feature_names=feature_name,
    max_depth=3,
    threshold=0.05
)
dfs(root)
bfs(root)

feature: Ear shape

feature: Face shape
feature: Whiskers

result: 1
result: 0
result: 1
result: 0



# Cat classification example
with one-hot encoding

| Pointy ears | Floppy ears | Round ears | Face shape | Whiskers | Cat  |
| ----------- | ----------- | ---------- | ---------- | -------- | ---- |
| 1           | 0           | 0          | 1          | 1        | 1    |
| 0           | 0           | 1          | 0          | 1        | 1    |
| 0           | 0           | 1          | 1          | 0        | 0    |
| 1           | 0           | 0          | 0          | 1        | 0    |
| 0           | 0           | 1          | 1          | 1        | 1    |
| 1           | 0           | 0          | 1          | 0        | 1    |
| 0           | 1           | 0          | 0          | 0        | 0    |
| 0           | 0           | 1          | 1          | 0        | 1    |
| 0           | 1           | 0          | 1          | 0        | 0    |
| 0           | 1           | 0          | 1          | 0        | 0    |

In [6]:
data = np.array([
    [1, 0, 0, 1, 1, 1],
    [0, 0, 1, 0, 1, 1],
    [0, 0, 1, 1, 0, 0],
    [1, 0, 0, 0, 1, 0],
    [0, 0, 1, 1, 1, 1],
    [1, 0, 0, 1, 0, 1],
    [0, 1, 0, 0, 0, 1],
    [0, 0, 1, 1, 0, 1],
    [0, 1, 0, 1, 0, 0],
    [0, 1, 0, 1, 0, 0],
], dtype=int)
feature_name = [
    'Pointy ears',
    'Floppy ears',
    'Round ears',
    'Face shape',
    'Whiskers'
]
root = DecisionTreeNode(
    data=data,
    features=[1, 1, 1, 1, 1],
    feature_names=feature_name,
    max_depth=5,
    threshold=0.01
)
dfs(root)
bfs(root)

feature: Floppy ears

feature: Face shape
feature: Face shape

result: 0
result: 1
feature: Pointy ears
feature: Pointy ears

result: 1
feature: Whiskers
result: 0
result: 1

result: 1
result: 0

